# Headlines Receiver


In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
#from datetime import time 
import warnings
warnings.filterwarnings("ignore")
ek.set_app_key('###########') # replace with app key
from datetime import datetime

## Index Constituents

### Thomson Reuters

In [ ]:
# Get Dataframe of current S&P 500 Constituents
# Source: https://community.developers.refinitiv.com/questions/28646/historical-component-stocks-of-sp-500-index.html
#data_SPX= ek.get_data(['.SPX'], ['TR.IndexConstituentRIC','TR.IndexConstituentName'])

# Get Dataframe of current S&P 100 Constituents
#data_OEXA= ek.get_data(['.OEXA'], ['TR.IndexConstituentRIC','TR.IndexConstituentName'])


In [ ]:
# Convert tuple to dataframe
#df_SPX =  pd.DataFrame(data_SPX, columns =['Instrument', 'Constituent RIC', 'Constituent Name'])
#df_SPX =  pd.DataFrame(data_SPX[0],columns =['Instrument', 'Constituent RIC', 'Constituent Name'])
#df_SPX[:]

#df_OEXA =  pd.DataFrame(data_OEXA[0],columns =['Instrument', 'Constituent RIC', 'Constituent Name'])
#df_OEXA[:]

In [ ]:
# Store dataframe to excel
#df_SPX.to_excel("SPX_Constituents.xlsx")
#df_OEXA.to_excel("OEXA_Constituents.xlsx")

### WRDS

In [ ]:
# load WRDS constituents list
cols=["from","thru","co_conm","co_tic","co_cusip"]
df_SPX_hist=pd.read_excel("../../07_WRDS/1_S&P 500 Index constituents/6.1_SPX_Constituents_incl_filled_dates - corr WRDS error.xlsx",usecols=cols)
df_SPX_hist

In [ ]:
# list of dataframes
dfs = []  

#comp_list=df_SPX_hist.co_cusip.tolist()
comp_list=df_SPX_hist.co_cusip.tolist()
len(comp_list)

In [ ]:
# Get all RICS jointly from list
#rics = ek.get_symbology(comp_list,from_symbol_type="CUSIP", to_symbol_type="RIC")
#rics

for c in comp_list:
    r = ek.get_symbology(c,from_symbol_type="CUSIP", to_symbol_type="RIC")
    dfs.append(r)

rics = pd.concat(dfs)

print(rics)

In [ ]:
# Add RICS to WRDS list and save to excel
df_SPX_hist["RIC"]=rics.RIC.to_list()
df_SPX_hist.to_excel("../WRDS_SPX_constituent incl RICS.xlsx")

In [ ]:
df_SPX_hist[df_SPX_hist.RIC.isnull()]

## Receive News Headlines

In [ ]:
#filter= df_SPX["Constituent Name"].str.contains('APPLE')
#df_SPX['Constituent RIC'][filter]

In [ ]:
"""Get news articles - only receives news from NewsWire"""
# R:IBM.N AND Topic:COVID AND Language:LEN
# Receive topics as field
#df = ek.get_news_headlines('R:IBM.N AND Language:LEN', date_from = "2019-06-01", count=100)


In [ ]:
#df.tail()

In [ ]:
# import manually completed SPX constitutents list
df_const=pd.read_excel("../../07_WRDS/1_S&P 500 Index constituents/1_WRDS_SPX_constituent incl RICS_manually completed.xlsx")
df_const

In [ ]:
# format date columns
df_const["from"]=pd.to_datetime(df_const["from"],format='%Y%m%d').dt.normalize()
df_const["thru"]=pd.to_datetime(df_const["thru"],format='%Y%m%d').dt.normalize()
df_const

In [ ]:
# Set up requests
from random import random
import time

# Define start and end date

# Set up date variables
start_month=10
start_year= 2019

# Set up date variables
end_month=10
end_year= 2019


fixed_end_date= datetime(end_year, end_month, 15)
end_date =fixed_end_date

start_date = datetime(start_year, start_month, 15)

# replace start date < start date with start date 
df_const["from"]=[start_date if x < start_date else x for x in df_const["from"]]

# replace NaT in thru with end date
df_const["thru"]=[fixed_end_date if pd.isnull(x) else x for x in df_const["thru"]]

df_const
df_const.to_excel("Fill_missing_datesSPX_Constituents_incl_dates.xlsx")

In [ ]:
# read prepared file
df_const=pd.read_excel("Fill_missing_news_14-10-19-SPX_Constituents_incl_dates.xlsx")
df_const

In [ ]:
# format date columns
df_const["from"]=pd.to_datetime(df_const["from"],format='%Y%m%d').dt.normalize()
df_const["thru"]=pd.to_datetime(df_const["thru"],format='%Y%m%d').dt.normalize()
df_const

In [ ]:
# create datestring
datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y, %H_%M_%S")

#comp_list=['CHRW.OQ','AJG.N','CNP.N','AMCR.N','WM.N']
#comp_list=['AAPL.OQ','CHRW.OQ']


collect_headlines = []
success= False
comp_loop_count=0
request_loop_count=0


# loop through list of companies
for index,row in df_const[:].iterrows():
    
    # set end date to row end date
    end_date= row["thru"]
    
    while success==False:    
        
        try:
            
            # receive dataframe of news
            df = ek.get_news_headlines('R:'+str(row["RIC"])+' AND Language:LEN', \
                                       date_from = str(row["from"]), date_to=str(end_date), count=100)
            request_loop_count+=1
            time.sleep(random() * 10 * 1)
            
            # set RIC to retrieved company
            df["RIC"]= row["RIC"]
            
        
            # append dataframe to list of dataframes
            collect_headlines.append(df)
            
            # if empty dataframe is received or end date = last date of retrieved dataframe
            if (df.shape[0]==0) or (end_date == df.tail(1).index.item()):
                success=True
            else:
                # set index of last retrieved data as end date
                end_date = df.tail(1).index.item()
            
        except Exception as exc:
                print(f"Encountered an unknown error: {exc}")
                
                # set RIC to retrieved company
                df["RIC"]= row["RIC"]
                
                # append dataframe to list of dataframes
                collect_headlines.append(df)
                
                # concat all dataframes into one
                df_headlines = pd.concat(collect_headlines)
                
                # store in csv
                df_headlines.to_csv(str(start_date.date())+ "_to_" + str(end_date.date()) + "_time_"+ datestring_time + "_headlines.csv")
                print("Last RIC: "+ str(row["RIC"]) +" Last enddate: "+str(end_date)+ " Error at: "+str(datetime.now()))
                raise
        
    # reset end date and success
    #end_date=fixed_end_date   
    success=False
    
    # Count iterations
    comp_loop_count+=1
    print(f"Done with: number: {index},total loops: {request_loop_count}")

# concat all dataframes into one
df_headlines = pd.concat(collect_headlines)

# store in csv
df_headlines.to_csv(str(start_date.date())+ "_to_"+ str(fixed_end_date.date())+ "_time_" + datestring_time + "_headlines.csv")
print(f"Done, requests sent: {request_loop_count} time: {datetime.now()}")